In [1]:
import sys, os, random, shutil

import numpy as np
from tqdm  import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split

# sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))))
sys.path.append('../')
sys.path.append('../../')


In [2]:
# df = np.load('../../ds/1000/boards.npy')
# label = np.load('../../ds/1000/scores.npy')
df = None
label = None
for i in range(5, 17, 1):
    new_df = np.load(f'../../ds/Session {i}/boards.npy')
    new_label = np.load(f'../../ds/Session {i}/scores.npy')

    if df is None:
        df = new_df
        label = new_label
    else:
        df = np.concatenate((df, new_df))
        label = np.concatenate((label, new_label))

# df = np.load('boards.npy')
# label = np.load('scores.npy')

print(df.shape)
print(label.shape)

(12000, 4, 5, 5)
(12000,)


In [3]:
df = np.transpose(df, (0, 2, 3, 1))

In [4]:
print(df.shape)

(12000, 5, 5, 4)


In [5]:
X_train, X_val = train_test_split(df, test_size=0.2, shuffle=False, random_state=0)
X_val, X_test = train_test_split(X_val, test_size=0.5, shuffle=False, random_state=0)

y_train, y_val = train_test_split(label, test_size=0.2, shuffle=False, random_state=0)
y_val, y_test = train_test_split(y_val, test_size=0.5, shuffle=False, random_state=0)

In [6]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(9600, 5, 5, 4)
(1200, 5, 5, 4)
(1200, 5, 5, 4)
(9600,)
(1200,)
(1200,)


In [7]:
class MyActivationLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(MyActivationLayer, self).__init__()

    def call(self, inputs):
        return tf.minimum(1.0, tf.maximum(0.0,inputs))



In [8]:
def build_model():
    # #Skip Connection
    inputs = tf.keras.layers.Input(shape=(5, 5, 4))

    #Input
    #Block 1
    x = tf.keras.layers.Conv2D(filters= 64, 
                               kernel_size=(2, 2),
                               padding='same',
                               activation='relu')(inputs)  
    x = tf.keras.layers.Conv2D(filters= 64, 
                               kernel_size=(2, 2),
                               padding='same',
                               activation='relu')(inputs)  
    x = tf.keras.layers.Conv2D(filters= 64, 
                               kernel_size=(2, 2),
                               padding='same',
                               activation='relu')(inputs)                                                             

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)   
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)  
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    return tf.keras.Model(inputs=inputs, outputs= outputs)

model = build_model()

In [9]:
import tensorflow.keras.callbacks as callbacks

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 5, 4)]         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 64)          1088      
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                102464    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0     

In [10]:
model.fit(x=X_train, y=y_train,
          batch_size=64,
          epochs=200,
          verbose=1,
          validation_data=(X_val, y_val),
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=5e-4)])
          # callbacks=[callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=5e-4)])

model.save('model.h5')

Epoch 1/200
150/150 [==============================] - 3s 6ms/step - loss: 0.0885 - root_mean_squared_error: 0.2974 - val_loss: 0.0774 - val_root_mean_squared_error: 0.2782 - lr: 0.0010
Epoch 2/200
150/150 [==============================] - 1s 4ms/step - loss: 0.0786 - root_mean_squared_error: 0.2804 - val_loss: 0.0756 - val_root_mean_squared_error: 0.2750 - lr: 0.0010
Epoch 3/200
150/150 [==============================] - 1s 5ms/step - loss: 0.0757 - root_mean_squared_error: 0.2752 - val_loss: 0.0757 - val_root_mean_squared_error: 0.2751 - lr: 0.0010
Epoch 4/200
150/150 [==============================] - 1s 4ms/step - loss: 0.0740 - root_mean_squared_error: 0.2721 - val_loss: 0.0741 - val_root_mean_squared_error: 0.2722 - lr: 0.0010
Epoch 5/200
150/150 [==============================] - 1s 4ms/step - loss: 0.0725 - root_mean_squared_error: 0.2693 - val_loss: 0.0768 - val_root_mean_squared_error: 0.2772 - lr: 0.0010
Epoch 6/200
150/150 [==============================] - 1s 4ms/step - l

In [11]:
def evaluate(X_df, y_df, threshold = 0.25):
    size = len(y_df)
    result = np.abs(model.predict(X_df)[0] - y_df)
    # print(f"Accuracy: {int(len(result[result < threshold])/size * 100)}%", )
    return int(len(result[result < threshold])/size * 100)

def statistic_model(X_df, y_df):
    result = dict({})
    for i in range(2, 21, 1):
        # print("======================================")
        # print(f"Threshold {i/20}")
        result[i/20] = (evaluate(X_df, y_df, i/20))
    return result

In [12]:
print(f"Accuracy: {evaluate(X_test, y_test, 0.3)}%")


38/38 [==============================] - 0s 2ms/step
Accuracy: 53%


In [13]:
print(statistic_model(X_test, y_test))

38/38 [==============================] - 0s 2ms/step
{0.1: 19, 0.15: 35, 0.2: 44, 0.25: 51, 0.3: 53, 0.35: 61, 0.4: 79, 0.45: 82, 0.5: 85, 0.55: 85, 0.6: 86, 0.65: 100, 0.7: 100, 0.75: 100, 0.8: 100, 0.85: 100, 0.9: 100, 0.95: 100, 1.0: 100}


In [14]:
print(statistic_model(X_train, y_train))

300/300 [==============================] - 1s 2ms/step
{0.1: 14, 0.15: 16, 0.2: 20, 0.25: 25, 0.3: 25, 0.35: 30, 0.4: 38, 0.45: 44, 0.5: 55, 0.55: 55, 0.6: 61, 0.65: 68, 0.7: 74, 0.75: 79, 0.8: 79, 0.85: 82, 0.9: 84, 0.95: 86, 1.0: 100}


In [15]:
print(np.abs(model.predict(X_train)))
print(y_train)

300/300 [==============================] - 1s 2ms/step
[[0.99985224]
 [0.51104236]
 [0.18023454]
 ...
 [0.51104236]
 [0.3638731 ]
 [0.8726169 ]]
[1.     0.3125 0.1875 ... 0.5    0.3125 0.875 ]
